# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. 

В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. 

**Цель** - построить модель для определения стоимости автомобиля. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [ ]:
#загружаем необходимые библиотеки
!pip install skimpy
!pip install catboost
!pip install lightgbm
!pip install sklearn

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

from catboost import CatBoostRegressor

from lightgbm import LGBMRegressor

from skimpy import clean_columns


In [ ]:
RANDOM = 123

warnings.filterwarnings('ignore')
model_list = []
rmse_score_list = []
best_params = []

In [ ]:
#загружаем датасет
try:
    data = pd.read_csv('/datasets/autos.csv')
except:
    data = pd.read_csv('/Users/amirk/Downloads/autos.csv')

### Анализ данных

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.describe(include='object')

In [ ]:
# приведем названия столбцов к стандартному виду
data = clean_columns(data)

In [ ]:
#сразу обращает на себя внимание столбец nimber_of_pictures, все значения равны 0. 
#Столбец неинформативный, если нет возможности восстановить данные, то удаляем его.
data.drop('number_of_pictures', axis=1, inplace=True)

In [ ]:
data.info()

In [ ]:
# найдем и удалим явные дубликаты
data.duplicated().sum()

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
# переведем столбцы с датами в datetime формат

data.loc[:, ['date_crawled', 'date_created', 'last_seen']] = pd.to_datetime \
(data.loc[:, ['date_crawled', 'date_created', 'last_seen']].stack()).unstack()

Пройдемся по всем столбцам, просмотрим выбросы и анамалии

In [ ]:
data['registration_year'].sort_values().unique()

Год регистрации не может быть равен 1000, 3000, 9999 и т.д. Это либо ошибка сбора информации, либо заглушки, о значении которых нужно узнавать у заказчика. На данный момент удалим  указанные значения

In [ ]:
data.query('1910 <= registration_year <= 2020', inplace=True)

In [ ]:
data['registration_year'].sort_values().unique()

Года имеют правильную размерность!

In [ ]:
data['registration_month'].value_counts()

Номер месяца не может быть равен 0, вероятно опять какая-то ошибка или заглушка, нужно уточнять. Пока заменим данные значения на первый месяц, т.к. более значимый признак это год и удалять 10% выборки не совсем верно.

In [ ]:
data['registration_month'].replace({0:1}, inplace=True)

In [ ]:
data['registration_month'].value_counts()

Месяца имеют правильные номера!

Создадим столбец с датой регистрации автомобиля. Дата регистрации автомобиля должна быть раньше, чем дата регистрации анкеты и даты выгрузки объявления. Проверим везде ли это соблюдено или есть ошибки. Если есть ошибки, то удалим их.

In [ ]:
# создадим столбец с датой регистрации автомобиля
data['date_registration'] = pd.to_datetime(dict(year=data['registration_year'], 
                                                month=data['registration_month'], 
                                                day=np.ones(len(data))))

In [ ]:
data.head()

In [ ]:
data.info()

Столбец добавлен!

In [ ]:
# Удалим ошибки
data.query('date_registration <= date_created <= date_crawled', inplace=True)

In [ ]:
data.info()

Ошибки удалены!

In [ ]:
data['date_crawled'].sort_values()

Аномалий нет!

In [ ]:
data['date_created'].sort_values()

Аномалий нет!

In [ ]:
data['price'].hist(bins=100)
plt.xlabel('Цена')
plt.ylabel('Количество объявлений')
plt.title('Распределение цены')
plt.show()

In [ ]:
data[data['price'] == 0]['price'].count()

Цена вряд ли может быть равна 0,вероятно это ошибка сбора информации. Удалим данные строки.

In [ ]:
data = data.query('price > 0')

In [ ]:
plt.barh(data['vehicle_type'].value_counts().index, data['vehicle_type'].value_counts())
plt.xlabel('Количество объявлений')
plt.ylabel('Тип кузова')
plt.title('Распределение различных типов кузова')
plt.show()

Сделаем предположение, что пропуски в vehicle_type можно найти по **brand, model, power, gearbox, fuel_type**. Посмотрим, автомобили с одинаковыми характеристиками имели только один тип кузова или несколько

In [ ]:
data.loc[:, ['brand', 'model','power', 'gearbox', 'fuel_type', 'vehicle_type']]. \
groupby(['brand', 'model', 'power','gearbox', 'fuel_type'], as_index=False).agg('nunique')


Постараемся заполнить vehicle_type в тех случаях, где был только один тип кузова. На примере данного параметра создадим функцию для заполнения пропусков в категориальных признаках. 

In [ ]:
def fill_col(data, features, target): 
    # надем модели авто, где только 1 тип кузова
    group_table1 = data.loc[:, features + [target]].groupby(features, as_index=False).agg('nunique').query(target + ' == 1')
    # найдем все модели авто  со всеми возможными типами кузова
    group_table = data.loc[:, features + [target]].groupby(features + [target], as_index=False).agg('nunique')
    # смерджим таблички 
    merge_table = group_table1.loc[:, features].merge(group_table, how='left', left_on=features, right_on=features)
    # создадим столбец в датасете, по которому мы будем заполнять пропуски в vehicle_type
    data['col'] = data[features].merge(merge_table, how='left', on=features)[target]
    # заполним пропуски, удалим созданный столбец
    data[target].fillna(data['col'], inplace=True)
    # оставшиеся пропуски заполним заглушкой
    data[target].fillna('undef', inplace=True)
    # удалим созданный столбец
    data.drop('col', axis=1, inplace=True)
    return data

In [ ]:
data = fill_col(data, ['brand', 'model','power', 'gearbox', 'fuel_type'], 'vehicle_type')

In [ ]:
data.info()

In [ ]:
plt.barh(data['gearbox'].value_counts().index, data['gearbox'].value_counts())
plt.xlabel('Количество объявлений')
plt.ylabel('Тип коробки передач')
plt.title('Распределение различных типов коробки передач')
plt.show()

Заполним пропуски!

In [ ]:
data = fill_col(data, ['brand', 'model','power', 'vehicle_type', 'fuel_type'], 'gearbox')

In [ ]:
data.info()

In [ ]:
data['model'].unique()

Неявных дубликатов нет, заполним пропуски!

In [ ]:
data = fill_col(data, ['brand', 'power', 'vehicle_type', 'fuel_type', 'gearbox'], 'model')

In [ ]:
data.info()

In [ ]:
data['power'].value_counts()

Нулевых значений в показателе мощности быть не может, поэтому заменим нули медианами по различным типам авто. 

In [ ]:
def fill_col_median(data, features, target):    
    group_table = data.loc[:, features + [target]].groupby(features, as_index=False).median()
    data['col'] = data[features].merge(group_table, how='left', on=features)[target]
    data['power'].replace({0:np.nan}, inplace=True) 
    data[target].fillna(data['col'], inplace=True)
    data.dropna(subset=target, inplace=True)
    data.drop('col', axis=1, inplace=True)
    return data

In [ ]:
data = fill_col_median(data, ['brand', 'model', 'vehicle_type', 'fuel_type', 'gearbox'], 'power')

In [ ]:
data.info()

Нули заполнить удалось!

In [ ]:
data['kilometer'].unique()

In [ ]:
data['kilometer'].hist(bins=13)
plt.xlabel('Пробег в километрах')
plt.ylabel('Количество объявлений')
plt.title('Распределение пробега автомобилей')
plt.show()

Выбросов и пропусков нет, данные имеют адекватные значения. 

In [ ]:
data['fuel_type'].unique()

Заполним пропуски!

In [ ]:
data = fill_col(data, ['brand', 'model', 'power', 'vehicle_type', 'gearbox'], 'fuel_type')

In [ ]:
data.info()

In [ ]:
data['brand'].unique()

Неявных дубликатов нет, пропусков нет.

In [ ]:
data['repaired'].unique()

In [ ]:
# Заменим пропуски на 'undef'
data['repaired'].fillna('undef', inplace=True)

In [ ]:
data.info()

In [ ]:
data['postal_code'].sort_values().unique()

Значение почтового индекса не совсем стандартное, выяснить у заказчика - ошибка это или нет. Пока оставляем!

In [ ]:
data['last_seen'].sort_values().unique()

In [ ]:
# Проверим и удалим при наличии случаи, когда последняя активность пользователя была раньше даты создания анкеты
data.query('last_seen >= date_created', inplace=True)

In [ ]:
data.info()

### Подготовка данных к загрузке в модели

- выделим target и features
- необходимо закодировать категориальные переменные, так как перед нами стоит задача регрессии. Применим OneHotEncoder
- выделим обучающую и тестовую выборки. Валидационную не будем выделять, потому что будем использовать кросс-валидацию (RandomizedSearchCV)

In [ ]:
y = data['price']
X = data.drop(data.select_dtypes('datetime64[ns]').columns.to_list() + ['price', 'model'], axis=1)
# пришлось удалить важный признак model, потому что возникала ошибка нехватки памяти!!!

obj_columns = X.select_dtypes('object').columns

ohe = OneHotEncoder(sparse=False, drop='first')
X_ohe = ohe.fit_transform(X[obj_columns])
X[ohe.get_feature_names()] = pd.DataFrame(X_ohe, columns=ohe.get_feature_names())
X.drop(obj_columns, axis=1, inplace=True)
X.fillna(0, inplace=True) #чтобы не выдавал ошибок для моделей дерево решений и случайный лес

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM)

## Обучение моделей

### LinearRegression model

In [ ]:
def best_model_lr(X_train, y_train):
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)
    model = LinearRegression()
    model.fit(X_train, y_train)
    prediction = model.predict(X_valid)
    model_list.append(model.__class__.__name__)
    rmse_score_list.append(mean_squared_error(y_valid, prediction, squared=False))
    best_params.append(model.get_params)
    print('Значение RMSE = ', mean_squared_error(y_valid, prediction, squared=False))
    return model 

In [ ]:
%%time
lr_model = best_model_lr(X_train, y_train)

In [ ]:
%%time
lr_model.predict(X_train)

### DecisionTreeRegressor model

In [ ]:
def best_model_dtr(X_train, y_train):
    model = DecisionTreeRegressor(random_state=RANDOM)
    params = {'max_depth':range(1,10), 'min_samples_split':range(2,10), 'min_samples_leaf':range(1,10)}
    grid_model = RandomizedSearchCV(model, 
                                    params, 
                                    cv=5, 
                                    scoring='neg_root_mean_squared_error', 
                                    n_jobs=-1, 
                                    verbose=1, 
                                    random_state=RANDOM)
    grid_model.fit(X_train, y_train)
    model_list.append(model.__class__.__name__)
    rmse_score_list.append(abs(grid_model.best_score_))
    best_params.append(grid_model.best_params_)
    print('Значение RMSE = ', abs(grid_model.best_score_))
    print('Лучшие параметры модели:', grid_model.best_params_)
    return grid_model

In [ ]:
%%time
dtr_model = best_model_dtr(X_train, y_train)

In [ ]:
%%time
dtr_model.predict(X_train)

### RandomForestRegressor model

In [ ]:
# Для начала найдем лучшие параметры модели на небольшом количестве деревьев 
#с целью экономии времени обучения, а в целом можно и нужно количество деревьев сразу смотреть.
# Затем подберем такой гиперпараметр как количество деревьев
def best_model_rfc(X_train, y_train):
    model = RandomForestRegressor(random_state=123, n_estimators=10)
    params = {'max_depth':range(1,10), 'min_samples_split':range(2,5), 'min_samples_leaf':range(1,5)}
    grid_model = RandomizedSearchCV(model, 
                                    params, 
                                    cv=5, 
                                    scoring='neg_root_mean_squared_error', 
                                    n_jobs=-1, 
                                    verbose=1, 
                                    random_state=RANDOM)
    grid_model.fit(X_train, y_train)
    model = RandomForestRegressor(random_state=123)
    params = {'n_estimators':range(20,200,30), 
              'max_depth':[grid_model.best_params_['max_depth']],
              'min_samples_split':[grid_model.best_params_['min_samples_split']],
              'min_samples_leaf':[grid_model.best_params_['min_samples_leaf']]}
    grid_model = RandomizedSearchCV(model, 
                                    params, 
                                    cv=5, 
                                    scoring='neg_root_mean_squared_error', 
                                    n_jobs=-1, 
                                    verbose=1, 
                                    random_state=RANDOM)
    grid_model.fit(X_train, y_train)
    model_list.append(model.__class__.__name__)
    rmse_score_list.append(abs(grid_model.best_score_))
    best_params.append(grid_model.best_params_)
    print('Значение RMSE = ', abs(grid_model.best_score_))
    print('Лучшие параметры модели:', grid_model.best_params_)
    return grid_model

In [ ]:
%%time
rfc_model = best_model_rfc(X_train, y_train)

In [ ]:
%%time
rfc_model.predict(X_train)

### LGBMRegresssor model

In [ ]:
def best_model_lgbm(X_train, y_train):
    model = LGBMRegressor(random_state=RANDOM)
    params = {'max_depth':range(1,10), 'min_data_in_leaf':range(1,20)}
    grid_model = RandomizedSearchCV(model, 
                                    params, 
                                    cv=5, 
                                    scoring='neg_root_mean_squared_error', 
                                    n_jobs=-1, 
                                    verbose=1, 
                                    random_state=RANDOM)
    grid_model.fit(X_train, y_train)
    model_list.append(model.__class__.__name__)
    rmse_score_list.append(abs(grid_model.best_score_))
    best_params.append(grid_model.best_params_)
    print('Значение RMSE = ', abs(grid_model.best_score_))
    print('Лучшие параметры модели:', grid_model.best_params_)
    return grid_model

In [ ]:
%%time
lgbm_model = best_model_lgbm(X_train, y_train)

In [ ]:
%%time
lgbm_model.predict(X_train)

### CatBoostRegressor model

In [ ]:
def best_model_cbr(X_train, y_train):
    model = CatBoostRegressor(random_state=RANDOM)
    params = {'max_depth':range(2,10), 'iterations':range(20,100, 20)}
    grid_model = RandomizedSearchCV(model, 
                                    params, 
                                    cv=5, 
                                    scoring='neg_root_mean_squared_error', 
                                    n_jobs=-1, 
                                    verbose=0, 
                                    random_state=RANDOM)
    grid_model.fit(X_train, y_train)
    model_list.append(model.__class__.__name__)
    rmse_score_list.append(abs(grid_model.best_score_))
    best_params.append(grid_model.best_params_)
    print('Значение RMSE = ', abs(grid_model.best_score_))
    print('Лучшие параметры модели:', grid_model.best_params_)
    return grid_model

In [ ]:
%%time
cbr_model = best_model_cbr(X_train, y_train)

In [ ]:
%%time
cbr_model.predict(X_train)

In [ ]:
# создадим сводную таблицу по результатам теста
top_list_df = pd.DataFrame({'Model':model_list, 
                            'RMSE':rmse_score_list,
                            'Best parameters':best_params }).sort_values(by='RMSE', ascending=True).reset_index(drop=True)
top_list_df

## Анализ моделей

В ходе исследования нами было оценено 5 моделей для прогнозирования цены продаваемого автомобиля. Наилучшие показатели (время обучения, время предсказания и качество предсказания) оказались у модели CatBoostRegressor. Именно данная модель будет применяться для прогнозирования цены!

| Модель                | Время обучения      | Время предсказания | RMSE |
| :---:                 |    :----:           |          :---:     | :---:|
| LinearRegression      | 1.6 s              | 109 ms            | 3780 |
| DecisionTreeRegressor | 33.6 s              | 172 ms            | 2259 |
| RandomForestRegressor | 30 min 49 s         | 5 s             | 2219 |
| LGBMRegressor     | 50 s          | 541 ms         | 2174 |
| **CatBoostRegressor**     | **64 s**              | **109 ms**            | **2167** |     



Проверим работу модели на тестовых данных!

In [ ]:
prediction = cbr_model.predict(X_test)
final_rmse = mean_squared_error(y_test, prediction, squared=False)
final_rmse

Лучшей моделью для прогнозирования цены автомобиля признана модель CatBoostRegressor c гиперпараметрами max_depth = 9, iterations = 80. Итоговое значение RMSE на тестовых данных составило 2178. Модель успешно прошла тестирование!

## Выводы

Перед нами стояла задача разработки модели, прогнозирующей рыночную цену автомобиля на основе технических характеристик, комплектации и других признаков. 

Заказчику были важны важны:
- качество предсказания;
- скорость предсказания;
- время обучения.

В ходе подготовки данных было выполнено:
- загрузка данных;
- удаление дубликатов;
- переопределение типов данных в столбцах;
- создание новых признаков;
- обработка выбросов, пропусков, аномалий; 
- кодировка категориальных признаков;
- разделение данных на тренировочную и тестовую выборки;
- обучение моделей

В ходе исследования нами было оценено 5 моделей для прогнозирования цены продаваемого автомобиля. Наилучшие показатели (время обучения, время предсказания и качество предсказания) оказались у модели CatBoostRegressor. Именно данная модель будет применяться для прогнозирования цены!

| Модель                | Время обучения      | Время предсказания | RMSE |
| :---:                 |    :----:           |          :---:     | :---:|
| LinearRegression      | 1.6 s              | 109 ms            | 3780 |
| DecisionTreeRegressor | 33.6 s              | 172 ms            | 2259 |
| RandomForestRegressor | 30 min 49 s         | 5 s             | 2219 |
| LGBMRegressor     | 50 s          | 541 ms         | 2174 |
| **CatBoostRegressor**     | **64 s**              | **109 ms**            | **2167** |  


Лучшей моделью для прогнозирования цены автомобиля признана модель CatBoostRegressor c гиперпараметрами max_depth = 9, iterations = 80. Итоговое значение RMSE на тестовых данных составило 2178. Модель успешно прошла тестирование!